In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import torch

from src import norm_functions, nn, nn_batch, nn_stratified, train_model

In [4]:
torch.manual_seed(16)

In [1]:
no_sessions = 3
no_participants = 15
no_channels = 62
no_features = 5

## <a id='two_cat'>1. Two categories: Positive and negative</a> 

In [5]:
no_videos = 10

#### Labels

In [6]:
labels_ = np.load('./data/emotion_labels.npy')
print(labels_.shape)

(675,)


In [7]:
participants_sessions_vector_ = np.load('./data/participants_sessions_vector.npy')
print(participants_sessions_vector_.shape)

(675,)


In [8]:
index_two_classes = []
for i in range(len(labels_)):
    if labels_[i]!=1:
        index_two_classes.append(i)
print(np.shape(index_two_classes))

(450,)


In [9]:
labels = labels_[index_two_classes]
print(labels.shape)

participants_sessions_vector = participants_sessions_vector_[index_two_classes]
print(participants_sessions_vector.shape)

(450,)
(450,)


### <a id='two_cat_batch'>1.1. Two categories: NN with batch normalization</a> 

#### Features

In [10]:
bandpower_SEED_ = np.load('./data/bandpower_SEED_multitaper.npy')
bandpower_SEED = bandpower_SEED_[index_two_classes]
bandpower_SEED = norm_functions.normalization(bandpower_SEED, no_videos=10)
print(bandpower_SEED.shape)

(450, 248)


#### Cross-subject NN

In [11]:
no_videos_session = no_sessions * no_videos
nn_nonorm = []
nn_norm = []

for i in range(no_participants):

    if i == 0:
        train_x_cross_subject = bandpower_SEED[1 * no_videos_session :]
        val_x_cross_subject = bandpower_SEED[0 : 1 * no_videos_session]

        train_y_cross_subject = labels[1 * no_videos_session :]
        val_y_cross_subject = labels[0 : 1 * no_videos_session]

        train_i_cross_subject = participants_sessions_vector[1 * no_videos_session :]
        val_i_cross_subject = participants_sessions_vector[0 : 1 * no_videos_session]

    elif i == (no_participants - 1):
        train_x_cross_subject = bandpower_SEED[: 14 * no_videos_session]
        val_x_cross_subject = bandpower_SEED[14 * no_videos_session :]

        train_y_cross_subject = labels[: 14 * no_videos_session]
        val_y_cross_subject = labels[14 * no_videos_session :]

        train_i_cross_subject = participants_sessions_vector[: 14 * no_videos_session]
        val_i_cross_subject = participants_sessions_vector[14 * no_videos_session :]

    else:
        train_x_cross_subject = np.concatenate(
            (
                bandpower_SEED[0 : i * no_videos_session, :],
                bandpower_SEED[(i + 1) * no_videos_session :, :],
            )
        )
        val_x_cross_subject = bandpower_SEED[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

        train_y_cross_subject = np.concatenate(
            (labels[0 : i * no_videos_session], labels[(i + 1) * no_videos_session :])
        )
        val_y_cross_subject = labels[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

        train_i_cross_subject = np.concatenate(
            (
                participants_sessions_vector[0 : i * no_videos_session],
                participants_sessions_vector[(i + 1) * no_videos_session :],
            )
        )
        val_i_cross_subject = participants_sessions_vector[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

    # NN nonorm
    netNoNorm = nn.Net()
    ts_acc = train_model.train_model_cross_subject(
        model=netNoNorm,
        train_x=train_x_cross_subject,
        test_x=val_x_cross_subject,
        train_y=train_y_cross_subject,
        test_y=val_y_cross_subject,
        train_i=train_i_cross_subject,
        test_i=val_i_cross_subject,
        no_epochs=100,
        normalize=False,
    )
    nn_nonorm.append(ts_acc)

    # NN norm
    netNorm = nn_batch.NetBatchNorm()
    ts_acc = train_model.train_model_cross_subject(
        model=netNorm,
        train_x=train_x_cross_subject,
        test_x=val_x_cross_subject,
        train_y=train_y_cross_subject,
        test_y=val_y_cross_subject,
        train_i=train_i_cross_subject,
        test_i=val_i_cross_subject,
        no_epochs=100,
        normalize=True,
    )
    nn_norm.append(ts_acc)

    print("Participant ", (i + 1))
    print("NN no norm: ", nn_nonorm[i])
    print("NN norm: ", nn_norm[i], "\n")

nn_nonorm = np.array(nn_nonorm)
nn_norm = np.array(nn_norm)

Participant  1
NN no norm:  0.6000000238418579
NN norm:  0.8333333134651184 

Participant  2
NN no norm:  0.6333333253860474
NN norm:  0.800000011920929 

Participant  3
NN no norm:  0.7333333492279053
NN norm:  0.699999988079071 

Participant  4
NN no norm:  0.699999988079071
NN norm:  0.7333333492279053 

Participant  5
NN no norm:  0.7333333492279053
NN norm:  0.8999999761581421 

Participant  6
NN no norm:  0.8666666746139526
NN norm:  0.9333333373069763 

Participant  7
NN no norm:  0.800000011920929
NN norm:  0.8999999761581421 

Participant  8
NN no norm:  0.5
NN norm:  0.9666666388511658 

Participant  9
NN no norm:  0.8666666746139526
NN norm:  0.9333333373069763 

Participant  10
NN no norm:  0.6000000238418579
NN norm:  0.7666666507720947 

Participant  11
NN no norm:  0.7666666507720947
NN norm:  0.9333333373069763 

Participant  12
NN no norm:  0.5666666626930237
NN norm:  0.5666666626930237 

Participant  13
NN no norm:  0.6000000238418579
NN norm:  0.9333333373069763 

P

In [12]:
print('NN no norm mean: ', str(round(np.mean(nn_nonorm), 3)))
print('NN no norm std: ', str(round(np.std(nn_nonorm), 3)))
print('NN norm mean: ', str(round(np.mean(nn_norm), 3)))
print('NN norm std: ', str(round(np.std(nn_norm), 3)))

NN no norm mean:  0.711
NN no norm std:  0.118
NN norm mean:  0.847
NN norm std:  0.115


### <a id='two_cat_stratified'>1.2. Two categories: NN with stratified normalization</a> 

#### Features

In [13]:
bandpower_SEED_ = np.load('./data/bandpower_SEED_multitaper.npy')
bandpower_SEED = bandpower_SEED_[index_two_classes]
bandpower_SEED = norm_functions.normalization_per_participant_session(bandpower_SEED, no_videos=10)
print(bandpower_SEED.shape)

(450, 248)


#### Cross-subject NN

In [14]:
no_videos_session = no_sessions * no_videos
nn_nonorm = []
nn_norm = []

for i in range(no_participants):

    if i == 0:
        train_x_cross_subject = bandpower_SEED[1 * no_videos_session :]
        val_x_cross_subject = bandpower_SEED[0 : 1 * no_videos_session]

        train_y_cross_subject = labels[1 * no_videos_session :]
        val_y_cross_subject = labels[0 : 1 * no_videos_session]

        train_i_cross_subject = participants_sessions_vector[1 * no_videos_session :]
        val_i_cross_subject = participants_sessions_vector[0 : 1 * no_videos_session]

    elif i == (no_participants - 1):
        train_x_cross_subject = bandpower_SEED[: 14 * no_videos_session]
        val_x_cross_subject = bandpower_SEED[14 * no_videos_session :]

        train_y_cross_subject = labels[: 14 * no_videos_session]
        val_y_cross_subject = labels[14 * no_videos_session :]

        train_i_cross_subject = participants_sessions_vector[: 14 * no_videos_session]
        val_i_cross_subject = participants_sessions_vector[14 * no_videos_session :]

    else:
        train_x_cross_subject = np.concatenate(
            (
                bandpower_SEED[0 : i * no_videos_session, :],
                bandpower_SEED[(i + 1) * no_videos_session :, :],
            )
        )
        val_x_cross_subject = bandpower_SEED[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

        train_y_cross_subject = np.concatenate(
            (labels[0 : i * no_videos_session], labels[(i + 1) * no_videos_session :])
        )
        val_y_cross_subject = labels[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

        train_i_cross_subject = np.concatenate(
            (
                participants_sessions_vector[0 : i * no_videos_session],
                participants_sessions_vector[(i + 1) * no_videos_session :],
            )
        )
        val_i_cross_subject = participants_sessions_vector[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

    # NN nonorm
    netNoNorm = nn.Net()
    ts_acc = train_model.train_model_cross_subject(
        model=netNoNorm,
        train_x=train_x_cross_subject,
        test_x=val_x_cross_subject,
        train_y=train_y_cross_subject,
        test_y=val_y_cross_subject,
        train_i=train_i_cross_subject,
        test_i=val_i_cross_subject,
        no_epochs=100,
        normalize=False,
    )
    nn_nonorm.append(ts_acc)

    # NN norm
    netNorm = nn_stratified.NetStratifiedNorm()
    ts_acc = train_model.train_model_cross_subject(
        model=netNorm,
        train_x=train_x_cross_subject,
        test_x=val_x_cross_subject,
        train_y=train_y_cross_subject,
        test_y=val_y_cross_subject,
        train_i=train_i_cross_subject,
        test_i=val_i_cross_subject,
        no_epochs=100,
        normalize=True,
    )
    nn_norm.append(ts_acc)

    print("Participant ", (i + 1))
    print("NN no norm: ", nn_nonorm[i])
    print("NN norm: ", nn_norm[i], "\n")

nn_nonorm = np.array(nn_nonorm)
nn_norm = np.array(nn_norm)

Participant  1
NN no norm:  0.699999988079071
NN norm:  0.7666666507720947 

Participant  2
NN no norm:  0.8666666746139526
NN norm:  0.800000011920929 

Participant  3
NN no norm:  0.9666666388511658
NN norm:  0.9333333373069763 

Participant  4
NN no norm:  0.8999999761581421
NN norm:  0.8666666746139526 

Participant  5
NN no norm:  0.8999999761581421
NN norm:  0.9666666388511658 

Participant  6
NN no norm:  0.8333333134651184
NN norm:  0.8999999761581421 

Participant  7
NN no norm:  0.8333333134651184
NN norm:  0.9333333373069763 

Participant  8
NN no norm:  0.9666666388511658
NN norm:  1.0 

Participant  9
NN no norm:  0.8333333134651184
NN norm:  0.9666666388511658 

Participant  10
NN no norm:  0.9333333373069763
NN norm:  0.9666666388511658 

Participant  11
NN no norm:  1.0
NN norm:  1.0 

Participant  12
NN no norm:  0.699999988079071
NN norm:  0.800000011920929 

Participant  13
NN no norm:  0.8999999761581421
NN norm:  0.9333333373069763 

Participant  14
NN no norm:  0.

In [15]:
print('NN no norm mean: ', str(round(np.mean(nn_nonorm), 3)))
print('NN no norm std: ', str(round(np.std(nn_nonorm), 3)))
print('NN norm mean: ', str(round(np.mean(nn_norm), 3)))
print('NN norm std: ', str(round(np.std(nn_norm), 3)))

NN no norm mean:  0.876
NN no norm std:  0.091
NN norm mean:  0.916
NN norm std:  0.074


## <a id='three_cat'>2. Three categories: Positive, neutral and negative</a> 

In [16]:
no_videos = 15

#### Labels

In [17]:
labels = np.load('./data/emotion_labels.npy')
print(labels.shape)
print(labels[0:15])

(675,)
[2. 1. 0. 0. 1. 2. 0. 1. 2. 2. 1. 0. 1. 2. 0.]


In [18]:
participants_sessions_vector = np.load('./data/participants_sessions_vector.npy')
print(participants_sessions_vector.shape)

(675,)


### <a id='three_cat_batch'>2.1. Three categories: NN with batch normalization</a> 

#### Features

In [19]:
bandpower_SEED = np.load('./data/bandpower_SEED_multitaper.npy')
bandpower_SEED = norm_functions.normalization(bandpower_SEED, no_videos=15)
print(bandpower_SEED.shape)

(675, 248)


#### Cross-subject NN

In [20]:
no_videos_session = no_sessions * no_videos
nn_nonorm = []
nn_norm = []

for i in range(no_participants):

    if i == 0:
        train_x_cross_subject = bandpower_SEED[1 * no_videos_session :]
        val_x_cross_subject = bandpower_SEED[0 : 1 * no_videos_session]

        train_y_cross_subject = labels[1 * no_videos_session :]
        val_y_cross_subject = labels[0 : 1 * no_videos_session]

        train_i_cross_subject = participants_sessions_vector[1 * no_videos_session :]
        val_i_cross_subject = participants_sessions_vector[0 : 1 * no_videos_session]

    elif i == (no_participants - 1):
        train_x_cross_subject = bandpower_SEED[: 14 * no_videos_session]
        val_x_cross_subject = bandpower_SEED[14 * no_videos_session :]

        train_y_cross_subject = labels[: 14 * no_videos_session]
        val_y_cross_subject = labels[14 * no_videos_session :]

        train_i_cross_subject = participants_sessions_vector[: 14 * no_videos_session]
        val_i_cross_subject = participants_sessions_vector[14 * no_videos_session :]

    else:
        train_x_cross_subject = np.concatenate(
            (
                bandpower_SEED[0 : i * no_videos_session, :],
                bandpower_SEED[(i + 1) * no_videos_session :, :],
            )
        )
        val_x_cross_subject = bandpower_SEED[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

        train_y_cross_subject = np.concatenate(
            (labels[0 : i * no_videos_session], labels[(i + 1) * no_videos_session :])
        )
        val_y_cross_subject = labels[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

        train_i_cross_subject = np.concatenate(
            (
                participants_sessions_vector[0 : i * no_videos_session],
                participants_sessions_vector[(i + 1) * no_videos_session :],
            )
        )
        val_i_cross_subject = participants_sessions_vector[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

    # NN nonorm
    netNoNorm = nn.Net()
    ts_acc = train_model.train_model_cross_subject(
        model=netNoNorm,
        train_x=train_x_cross_subject,
        test_x=val_x_cross_subject,
        train_y=train_y_cross_subject,
        test_y=val_y_cross_subject,
        train_i=train_i_cross_subject,
        test_i=val_i_cross_subject,
        no_epochs=100,
        normalize=False,
    )
    nn_nonorm.append(ts_acc)

    # NN norm
    netNorm = nn_batch.NetBatchNorm()
    ts_acc = train_model.train_model_cross_subject(
        model=netNorm,
        train_x=train_x_cross_subject,
        test_x=val_x_cross_subject,
        train_y=train_y_cross_subject,
        test_y=val_y_cross_subject,
        train_i=train_i_cross_subject,
        test_i=val_i_cross_subject,
        no_epochs=100,
        normalize=True,
    )
    nn_norm.append(ts_acc)

    print("Participant ", (i + 1))
    print("NN no norm: ", nn_nonorm[i])
    print("NN norm: ", nn_norm[i], "\n")

nn_nonorm = np.array(nn_nonorm)
nn_norm = np.array(nn_norm)

Participant  1
NN no norm:  0.42222222685813904
NN norm:  0.6222222447395325 

Participant  2
NN no norm:  0.42222222685813904
NN norm:  0.5111111402511597 

Participant  3
NN no norm:  0.6222222447395325
NN norm:  0.7333333492279053 

Participant  4
NN no norm:  0.644444465637207
NN norm:  0.6000000238418579 

Participant  5
NN no norm:  0.4888888895511627
NN norm:  0.7555555701255798 

Participant  6
NN no norm:  0.7111111283302307
NN norm:  0.5777778029441833 

Participant  7
NN no norm:  0.35555556416511536
NN norm:  0.6666666865348816 

Participant  8
NN no norm:  0.4444444477558136
NN norm:  0.7333333492279053 

Participant  9
NN no norm:  0.7111111283302307
NN norm:  0.7333333492279053 

Participant  10
NN no norm:  0.46666666865348816
NN norm:  0.6888889074325562 

Participant  11
NN no norm:  0.6222222447395325
NN norm:  0.7333333492279053 

Participant  12
NN no norm:  0.46666666865348816
NN norm:  0.5333333611488342 

Participant  13
NN no norm:  0.42222222685813904
NN norm:

In [21]:
print('NN no norm mean: ', str(round(np.mean(nn_nonorm), 3)))
print('NN no norm std: ', str(round(np.std(nn_nonorm), 3)))
print('NN norm mean: ', str(round(np.mean(nn_norm), 3)))
print('NN norm std: ', str(round(np.std(nn_norm), 3)))

NN no norm mean:  0.535
NN no norm std:  0.113
NN norm mean:  0.662
NN norm std:  0.083


### <a id='three_cat_stratified'>2.2. Three categories: NN with stratified normalization</a> 

#### Features

In [22]:
bandpower_SEED = np.load('./data/bandpower_SEED_multitaper.npy')
bandpower_SEED = norm_functions.normalization_per_participant_session(bandpower_SEED, no_videos=15)
print(bandpower_SEED.shape)

(675, 248)


#### Cross-subject NN

In [23]:
no_videos_session = no_sessions * no_videos
nn_nonorm = []
nn_norm = []

for i in range(no_participants):

    if i == 0:
        train_x_cross_subject = bandpower_SEED[1 * no_videos_session :]
        val_x_cross_subject = bandpower_SEED[0 : 1 * no_videos_session]

        train_y_cross_subject = labels[1 * no_videos_session :]
        val_y_cross_subject = labels[0 : 1 * no_videos_session]

        train_i_cross_subject = participants_sessions_vector[1 * no_videos_session :]
        val_i_cross_subject = participants_sessions_vector[0 : 1 * no_videos_session]

    elif i == (no_participants - 1):
        train_x_cross_subject = bandpower_SEED[: 14 * no_videos_session]
        val_x_cross_subject = bandpower_SEED[14 * no_videos_session :]

        train_y_cross_subject = labels[: 14 * no_videos_session]
        val_y_cross_subject = labels[14 * no_videos_session :]

        train_i_cross_subject = participants_sessions_vector[: 14 * no_videos_session]
        val_i_cross_subject = participants_sessions_vector[14 * no_videos_session :]

    else:
        train_x_cross_subject = np.concatenate(
            (
                bandpower_SEED[0 : i * no_videos_session, :],
                bandpower_SEED[(i + 1) * no_videos_session :, :],
            )
        )
        val_x_cross_subject = bandpower_SEED[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

        train_y_cross_subject = np.concatenate(
            (labels[0 : i * no_videos_session], labels[(i + 1) * no_videos_session :])
        )
        val_y_cross_subject = labels[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

        train_i_cross_subject = np.concatenate(
            (
                participants_sessions_vector[0 : i * no_videos_session],
                participants_sessions_vector[(i + 1) * no_videos_session :],
            )
        )
        val_i_cross_subject = participants_sessions_vector[
            i * no_videos_session : (i + 1) * no_videos_session
        ]

    # NN nonorm
    netNoNorm = nn.Net()
    ts_acc = train_model.train_model_cross_subject(
        model=netNoNorm,
        train_x=train_x_cross_subject,
        test_x=val_x_cross_subject,
        train_y=train_y_cross_subject,
        test_y=val_y_cross_subject,
        train_i=train_i_cross_subject,
        test_i=val_i_cross_subject,
        no_epochs=100,
        normalize=False,
    )
    nn_nonorm.append(ts_acc)

    # NN norm
    netNorm = nn_stratified.NetStratifiedNorm()
    ts_acc = train_model.train_model_cross_subject(
        model=netNorm,
        train_x=train_x_cross_subject,
        test_x=val_x_cross_subject,
        train_y=train_y_cross_subject,
        test_y=val_y_cross_subject,
        train_i=train_i_cross_subject,
        test_i=val_i_cross_subject,
        no_epochs=100,
        normalize=True,
    )
    nn_norm.append(ts_acc)

    print("Participant ", (i + 1))
    print("NN no norm: ", nn_nonorm[i])
    print("NN norm: ", nn_norm[i], "\n")

nn_nonorm = np.array(nn_nonorm)
nn_norm = np.array(nn_norm)

Participant  1
NN no norm:  0.5777778029441833
NN norm:  0.7777777910232544 

Participant  2
NN no norm:  0.6666666865348816
NN norm:  0.5777778029441833 

Participant  3
NN no norm:  0.9111111164093018
NN norm:  0.9333333373069763 

Participant  4
NN no norm:  0.7777777910232544
NN norm:  0.8222222328186035 

Participant  5
NN no norm:  0.800000011920929
NN norm:  0.8444444537162781 

Participant  6
NN no norm:  0.6888889074325562
NN norm:  0.800000011920929 

Participant  7
NN no norm:  0.7555555701255798
NN norm:  0.800000011920929 

Participant  8
NN no norm:  0.9555555582046509
NN norm:  0.9111111164093018 

Participant  9
NN no norm:  0.7111111283302307
NN norm:  0.8444444537162781 

Participant  10
NN no norm:  0.6222222447395325
NN norm:  0.7111111283302307 

Participant  11
NN no norm:  0.8222222328186035
NN norm:  0.7555555701255798 

Participant  12
NN no norm:  0.6222222447395325
NN norm:  0.8666666746139526 

Participant  13
NN no norm:  0.6000000238418579
NN norm:  0.6222

In [24]:
print('NN no norm mean: ', str(round(np.mean(nn_nonorm), 3)))
print('NN no norm std: ', str(round(np.std(nn_nonorm), 3)))
print('NN norm mean: ', str(round(np.mean(nn_norm), 3)))
print('NN norm std: ', str(round(np.std(nn_norm), 3)))

NN no norm mean:  0.751
NN no norm std:  0.125
NN norm mean:  0.796
NN norm std:  0.104
